# Процессы Леви

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import scipy.interpolate as spint
%matplotlib inline

В этом ноутбуке мы поисследуем несколько примеров процессов Леви.

## Винеровский процесс

Винеровский процесс мы уже умеем симулировать, это настолько просто, что мы не будем ради этого создавать отдельный класс и скопируем функцию из предыдущих семинаров.

In [ ]:
def simulateWienerProcess(x0,ts,Ntraj):
    '''
    Simulates trajectories of Wiener process
    Input
    float x0 -- initial value
    float[] ts -- times
    int Ntraj -- number of trajectories to simulate
    Returns
    float[] of shape (Ntraj,xs.shape[0],len(ts))
    '''
    try:
        d=x0.shape[0]
    except:
        d=1

    xs=np.zeros([Ntraj,d,len(ts)])
    xs[...,0]=x0
    for kk in np.arange(1,len(ts)):
        noises=np.random.randn(Ntraj,d)
        xs[...,kk]=xs[...,kk-1] + np.sqrt(ts[kk]-ts[kk-1])*noises
    
    return xs

## Процесс Пуассона

Пуассоновский процесс -- это процесс Леви с Пуассоновскими приращениями:

$$
N_{t} - N_{s} \sim \Pi(\lambda(t-s)).
$$

Тем не менее, для симуляций удобнее всего семплировать времена прыжков $T_i$, что делается так:
1. Семплируются независимо интервалы между прыжками $\Delta_i \sim Exp(\lambda)$;
2. Время $T_k = \sum_{i=1}^k \Delta_i$.

В моменты $T_k$ значение процесса меняется с $k-1$ на $k$.

Пуассоновский процесс мы далее мы будем использовать как строительный блок для процессов Леви.

In [ ]:
class PoissonProcess:
    
    def __init__(self, lam):
        '''
        Parameters
        float lam -- intensity parameter lambda
        '''
        self.lam = lam
        
    def sampleTrajs(self, N, T):
        '''
        Samples N trajectories of the process on range [0,T]
    
        Parameters
        int N -- number of trajectories
        float T -- the observation time (all larger sampling times will be deleted)
    
        Returns
        list(list()) -- jump times
        '''
        intervals = #???   #add some heuristic 
        jumpTimes = #???
        
        times = [ np.concatenate([np.zeros([1]),intervals[i,np.where(intervals[i,:]>0)[0]]]) for i in np.arange(N)]

        return times
    
    def interpolateTimes(self, ts, times):
        '''
        Performs nearest const interpolation
        '''
        # don't forget bounds_error=False, fill_value="extrapolate" options in interpolation
        # use spint.interp1d
        #return ??
        pass

In [ ]:
lam=3
N=100
T=3
Nt=10
ts = np.arange(0,T+T/(2*Nt), T/Nt)

pois = PoissonProcess(lam)
times = pois.sampleTrajs(N,T)
trajs = pois.interpolateTimes(ts, times)

#print(trajs, times)


In [ ]:
f, ax = plt.subplots(figsize=(13,7))

ax.grid()
ax.set_title('Simulated trajectories of Poisson process',fontsize=16)
ax.set_xlabel('t, time',fontsize=16)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.step(ts,trajs.T,"--",where="post", linewidth=1.2, alpha=0.8)

## Сложенный процесс Пуассона

Сложенный процесс Пуассона обобщает идею Пуассоновского и считающего процесса, разрешая прыжки случайного размера и направления. Семплируется последовательность iid величин $Y_i$ из распределения $F$, параллельно независимо запускается Пуассоновский процесс $N_t$. Сложенный процесс получается как сумма случайного числа слагаемых
$$
X_t = \sum_{i=1}^{N_t} Y_i.
$$

Мы уже умеем семплировать времена прыжков из Пуассоновского процесса, остаётся досемплировать сами прыжки.

In [ ]:
class CompoundPoissonProcess:
    
    def __init__(self, lam, Fsampler):
        '''
        Parameters
        float lam -- intensity parameter of Poisson process
        obj Fsampler -- sampler function taking N and returning N iid samples
        '''
        self.lam = lam
        self.pois = PoissonProcess(self.lam)
        self.sampler = Fsampler
        
    def sampleTrajs(self,N,T):
        '''
        Samples N trajectories of the process on range [0,T]
    
        Parameters
        int N -- number of trajectories
        float T -- the observation time (all larger sampling times will be deleted)
    
        Returns
        list(list()), list(list()) -- jump times and values at jumps
        '''
        jumpTimes = #sample Poisson jump times
        values = #construct values at jump times
        return jumpTimes, values
    
    def interpolateTimes(self, ts, times, values):
        '''
        Performs nearest const interpolation
        '''
        #adjust interp1d from Poisson
        pass     

In [ ]:
mu=0.2
sigma=np.sqrt(2)
def normalSampler(N):
    return np.random.normal(loc=mu,scale=sigma,size=(N,))

lam = 2
cpois = CompoundPoissonProcess(lam=lam, Fsampler=normalSampler)

In [ ]:
N=3
T=4
times, values = cpois.sampleTrajs(N,T)
trajs = cpois.interpolateTimes(ts, times, values)

In [ ]:
f, ax = plt.subplots(figsize=(13,7))

ax.grid()
ax.set_title('Simulated trajectories of Compound Poisson process',fontsize=16)
ax.set_xlabel('t, time',fontsize=16)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.step(ts,trajs.T,"--",where="post", linewidth=1.2, alpha=0.8)

## Модель Мертона

Модель Мертона совершенствует модель Броуновского движения, добавляя в показатель экспоненты сложенный Пуассоновский процесс:

$$
X_t = x_0
$$

In [ ]:
class MertonProcess:
    
    def __init__(self,mu,sigma,muJump,sigmaJump, lam):
        self.mu = mu
        self.sigma = sigma        
        self.muJump = muJump
        self.sigmaJump = sigmaJump
        self.lam = lam
        
        def normalSampler(N):
            return np.random.normal(loc=self.mu,scale=self.sigma,size=(N,))
        self.cpois = CompoundPoissonProcess(self.lam,Fsampler=normalSampler)
    
    def sampleTrajs(self,x0,N,ts):
        '''
        Simulates trajectories of Merton model with parameters mu and sigma
        
        !Assuming ts[0]=0
        
        Input
        float[] x0 -- initial price
        float[] ts -- times
        int Ntraj -- number of trajectories to simulate
        Returns
        float[] of shape (Ntraj,len(ts)) , trajectories
        '''
        try:
                wienProcs = simulateWienerProcess(np.zeros(x0.shape),ts-ts[0],N)
        except:
                wienProcs = simulateWienerProcess(0,ts-ts[0],N)
                
        wienProcs = wienProcs[:,0,:]
        print(wienProcs.shape)
        cpoisTimes, cpoisValues = self.cpois.sampleTrajs(N,ts[-1])
        cpoisTrajs = self.cpois.interpolateTimes(ts,cpoisTimes,cpoisValues)
        print(cpoisTrajs.shape)
        #print(wienProcs.shape)
        xs = np.zeros( wienProcs.shape )
        xs[...,0] = x0       
        xs = x0 * np.exp((mu - sigma**2/2)*(ts-ts[0])[None,:] + sigma*wienProcs + cpoisTrajs)
        return xs


In [ ]:
mu=0.2
sigma=np.sqrt(2)
muJump = 0.2
sigmaJump = 0.2
def normalSampler(N):
    return np.random.normal(loc=mu,scale=sigma,size=(N,))

lam = 2
mert = MertonProcess(mu, sigma, muJump, sigmaJump, lam)

In [ ]:
N=10
T=4
Nt=200
ts = np.arange(0,T + T/(2*Nt), T/Nt)
x0=np.array([100])

trajs = mert.sampleTrajs(x0,N,ts)

In [ ]:
f, ax = plt.subplots(figsize=(13,7))

ax.grid()
ax.set_title('Simulated trajectories of Merton process',fontsize=16)
ax.set_xlabel('t, time',fontsize=16)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.plot(ts,trajs.T,"--", linewidth=1.2, alpha=0.8)
ax.set_yscale("log")